In [ ]:
!saved_model_cli show --dir "../input/baseline-landmark-retrieval-model/baseline_landmark_retrieval_model" --all

In [ ]:
import numpy as np
import os
import cv2
import glob

import tensorflow as tf
import keras
from keras.models import load_model, save_model
from keras.layers import Input, GlobalAveragePooling2D, GlobalMaxPooling2D
import keras.backend as K
from keras.models import Model, load_model
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

There are varying shapes of images as you can see below, meaning we'll need to resize images inside the model.

In [ ]:
files = glob.glob("../input/landmark-retrieval-2020/train/a/b/c/*.jpg")
for i in range(10):
    im = cv2.imread(files[i])
    print(im.shape)

Now let's load our model. In this case the vanilla VGG16 pretrained model of Keras for demonstration purposes. Since this is not trained on any retrieval dataset, the score will most probably be zero.

In [ ]:
backend, layers, models, keras_utils = tf.keras.backend, tf.keras.layers, tf.keras.models, tf.keras.utils

In [ ]:
def get_swish():

    def swish(x):
        """Swish activation function: x * sigmoid(x).
        Reference: [Searching for Activation Functions](https://arxiv.org/abs/1710.05941)
        """

        if backend.backend() == 'tensorflow':
            try:
                # The native TF implementation has a more
                # memory-efficient gradient implementation
                return backend.tf.nn.swish(x)
            except AttributeError:
                pass

        return x * backend.sigmoid(x)

    return swish


def get_dropout():
    """Wrapper over custom dropout. Fix problem of ``None`` shape for tf.keras.
    It is not possible to define FixedDropout class as global object,
    because we do not have modules for inheritance at first time.
    Issue:
        https://github.com/tensorflow/tensorflow/issues/30946
    """

    class FixedDropout(layers.Dropout):
        def _get_noise_shape(self, inputs):
            if self.noise_shape is None:
                return self.noise_shape

            symbolic_shape = backend.shape(inputs)
            noise_shape = [symbolic_shape[axis] if shape is None else shape
                           for axis, shape in enumerate(self.noise_shape)]
            return tuple(noise_shape)

    return FixedDropout

In [ ]:
model = tf.keras.models.load_model('../input/saving-model-iteration/saved_iteration_1.h5', compile=False, 
                                   custom_objects={'FixedDropout':get_dropout(),'swish':get_swish()})

In [ ]:
model.load_weights('../input/landmark-metric-learning/efficientnet-b0.h5')

In [ ]:
# backbone_name = 'efficientnet-b0'
# if backbone_name.startswith('efficientnet'):
#     model_fn = getattr(efn, f'EfficientNetB{backbone_name[-1]}')

In [ ]:
# # vgg = VGG16(input_shape=(224,224,3), weights=None, include_top=False)
# # vgg.load_weights("../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")

# # input_image = Input((224,224,3))
# # x = vgg(input_image)
# # output = GlobalMaxPooling2D()(x)

# # model = Model(inputs=[input_image], outputs=[output])
# # model.summary()
# model = build_model(engine=model_fn, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet') 


Now the main part! The *input_image* will be in it's own variable shape and hence we need to resize it within the model.

In [ ]:
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = model
    
    @tf.function(input_signature=[
      tf.TensorSpec(shape=[None, None, 3], dtype=tf.uint8, name='input_image')
    ])
    def call(self, input_image):
        output_tensors = {}
        
        # resizing
        im = tf.image.resize(input_image, [256,256])
        # preprocessing
        im = tf.cast(im, tf.float32)
        im = im / 255.0
        
        extracted_features = self.model(tf.convert_to_tensor([im], dtype=tf.float32))[0]
        output_tensors['global_descriptor'] = tf.identity(extracted_features, name='global_descriptor')
        return output_tensors

Now we create and save our model instance.

In [ ]:
m = MyModel() #creating our model instance

served_function = m.call
tf.saved_model.save(
      m, export_dir="./my_model", signatures={'serving_default': served_function})

In [ ]:
!ls ./my_model/variables

In [ ]:
from zipfile import ZipFile

with ZipFile('submission.zip','w') as zip:           
    zip.write('./my_model/saved_model.pb', arcname='saved_model.pb') 
    zip.write('./my_model/variables/variables.data-00000-of-00002', arcname='variables/variables.data-00000-of-00002')
    zip.write('./my_model/variables/variables.data-00001-of-00002', arcname='variables/variables.data-00001-of-00002') 
    zip.write('./my_model/variables/variables.index', arcname='variables/variables.index') 

Last but not the least, let's visualize our model to see if the structure is as per the requirements.

In [ ]:
!saved_model_cli show --dir ./my_model/ --all

Please upvote and let me know if this helps!